In [235]:
%pip install --upgrade pip
%pip install pandas
%pip install openpyxl
%pip install yfiles_jupyter_graphs
%pip install ipywidgets

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [236]:
import pandas as pd
import os
import json
import traceback
from yfiles_jupyter_graphs import GraphWidget
import ipywidgets as widgets

In [237]:
class DDFJson():
  
  def __init__(self):
    self.id_index = { 
      'code': 0, 
      'organisation': 0,
      'study_identifier': 0,
      'study_epoch': 0,
      'study_arm': 0,
      'study_cell': 0,
      'entry': 0, 'exit': 0, 'timepoint': 0, 'timeline': 0, 
      'timing': 0, 'study_design': 0, 'study': 0, 'activity': 0, 
      'encounter': 0, 'bc_surrogate': 0 
    }
    self.dicts = {}

  def increment_index(self, name):
    self.id_index[name] += 1

  def build_id(self, name):
    self.increment_index(name)
    return "%s_%s" % (name, self.id_index[name])

  def add_code(self, code, code_system, code_system_version, decode):
    id = self.build_id('code')
    result = { '_type': 'Code', 'codeId': id, 'code': code, 'codeSystem': code_system, 'codeSystemVersion': code_system_version, 'decode': decode }
    self.dicts[id] = result
    return result

  def add_organisation(self, scheme, identifier, name, type):
    id = self.build_id('organisation')
    result = { '_type': 'Organisation', 'organisationId': id, 'organisationIdentifierScheme': scheme, 'organisationIdentifier': identifier, 'organisationName': name, 'organisationType': type }
    self.dicts[id] = result
    return result

  def add_study_identifier(self, identifier, scope):
    id = self.build_id('study_identifier')
    result = { '_type': 'StudyIdentifier', 'studyIdentifierId': id, 'studyIdentifier': identifier, 'studyIdentifierScope': scope }
    self.dicts[id] = result
    return result

  def add_study_epoch(self, name, description):
    id = self.build_id('study_epoch')
    result = { '_type': 'StudyEpoch', 'studyEpochId': id, 'studyEpochName': name, 'studyEpochDescription': description, 'studyEpochType': "", 'previousStudyEpochId': "", 'nextStudyEpochId': "", 'encounters': [] }
    self.dicts[id] = result
    return result

  def add_study_arm(self, name, description):
    id = self.build_id('study_arm')
    result = { '_type': 'StudyArm', 'studyArmId': id, 'studyArmName': name, 'studyArmDescription': description, 'studyArmType': None, 'studyArmDataOriginDescription': "", 'studyArmDataOriginType': None }
    self.dicts[id] = result
    return result

  def add_study_cell(self, arm, epoch):
    id = self.build_id('study_cell')
    result = { '_type': 'StudyCell', 'studyCellId': id, 'studyArm': arm, 'studyEpoch': epoch, 'studyElements': [] }
    self.dicts[id] = result
    return result

  def add_entry(self, description, timepoint_id):
    id = self.build_id('entry')
    result = { '_type': 'Entry', 'entryId': id, 'entryDescription': description, 'nextTimepointId': timepoint_id }
    self.dicts[id] = result
    return result

  def add_exit(self):
    id = self.build_id('exit')
    result = { '_type': 'Exit', 'exitId': id }
    self.dicts[id] = result
    return result

  def add_timepoint(self, previous_timepoint_id, timing, activities, encounter):
    id = self.build_id('timepoint')
    result = { '_type': 'Timepoint', 'timepointId': id, 'nextTimepointId': None, 'scheduledAt': timing, 'timepointActivityIds': activities, 'timepointEncounterId': encounter }
    self.dicts[id] = result
    if not previous_timepoint_id == None:
      self.dicts[previous_timepoint_id]['nextTimepointId'] = id
    return result

  def add_previous_timing(self, value, relative_to_from, window, to_id):
    id = self.build_id('timing')
    result = { '_type': 'Timing', 'timingId': id, 'type': "after", 'value': value, 'relativeToFrom': relative_to_from, 'window': window, 'relativeTo': to_id }
    self.dicts[id] = result
    return result

  def add_next_timing(self, value, relative_to_from, window, to_id):
    id = self.build_id('timing')
    result = { '_type': 'Timing', 'timingId': id, 'type': "next", 'value': value, 'relativeToFrom': relative_to_from, 'window': window, 'relativeTo': to_id }
    self.dicts[id] = result
    return result

  def add_anchor_timing(self, value, cycle=""):
    id = self.build_id('timing')
    result = { '_type': 'Timing', 'timingId': id, 'type': "anchor", 'value': value, 'cycle': cycle, 'relativeToFrom': None, 'window': None, 'relativeTo': None }
    self.dicts[id] = result
    return result

  def add_condition_timing(self, value, to_id):
    id = self.build_id('timing')
    result = { '_type': 'Condition', 'conditionId': id, 'type': "condition", 'value': value, 'relativeToFrom': None, 'window': None, 'relativeTo': to_id }
    self.dicts[id] = result
    return result

  def add_cycle_start_timing(self, value):
    id = self.build_id('timing')
    result = { '_type': 'CycleStart', 'cycleStartId': id, 'type': "cycle start", 'value': value, 'relativeToFrom': None, 'window': None, 'relativeTo': None }
    self.dicts[id] = result
    return result

  def add_timeline(self, entry, timepoints, exit):
    id = self.build_id('timeline')
    result = { '_type': 'Timeline', 'timelineId': id, 'timelineEntry': entry, 'timelineTimepoints': timepoints, 'timelineExit': exit }
    self.dicts['id'] = result
    return result
  
  def add_activity(self, name, description, conditional=False, conditional_reason="", surrogates=[]):
    id = self.build_id('activity')
    result = { '_type': 'Activity', 'activityId': id, 'activityName': name, 'activityDescription': description, 'activityIsConditional': conditional, 'activityConditionalReason': conditional_reason, 'bcSurrogateIds': surrogates }
    self.dicts['id'] = result
    return result

  def add_encounter(self, name, description, enc_type, env_setting, contact_modes):
    id = self.build_id('encounter')
    result = { '_type': 'Encounter', 'encounterId': id, 'encounterName': name, 'encounterDescription': description, 'encounterType': enc_type, 'encounterEnvironmentalSetting': env_setting, 'encounterContactMode': contact_modes }
    self.dicts['id'] = result
    return result

  def add_biomedical_concept_surrogate(self, name, description, reference):
    id = self.build_id('bc_surrogate')
    result = { '_type': 'BCSurrogate', 'bcSurrogateId': id, 'bcSurrogateName': name, 'bcSurrogateDescription': description, 'bcSurrogateReference': reference }
    self.dicts['id'] = result
    return result

  def add_study_design(self, intent_types, trial_types, intervention_model, cells):
    #model, therapeutic_areas, cells, indications, objectives, populations, interventions, workflows, estimands, encounters, activities, surrogates
    id = self.build_id('study_design')
    result = { '_type': 'StudyDesign', 'studyDesignId': id, 
      'studyDesignName': "",
      'studyDesignDescription': "",
      'trialIntentTypes': intent_types,
      'trialType': trial_types,
      'interventionModel': intervention_model,
      'studyCells': cells,
      'studyIndications': [],
      'studyInvestigationalInterventions': [],
      'studyStudyDesignPopulations': [],
      'studyObjectives': [],
      'studyWorkflows': [],
      'therapeuticAreas': [],
      'studyEstimands': [],
      'encounters': [],
      'activities': [],
      'studyDesignRationale': "",
      'studyDesignBlindingScheme': None,
      'biomedicalConcepts': [],
      'bcCategories': [],
      'bcSurrogates': []
    }
    self.dicts['id'] = result
    return result
    
  def add_study(self, title, version, type, phase, ta, rationale, acronym, identifiers, protocols, designs):
    id = self.build_id('study')
    result = { '_type': 'Study', 'studyId': id, 'studyTitle': title, 'studyVersion': version, 'studyType': type, 'studyPhase': phase, 'businessTherapueticAreas': ta, 'studyRationale': rationale, 'studyAcronym': acronym, 'studyIdentifiers': identifiers, 'studyProtocolVersions': protocols, 'studyDesigns': designs }
    self.dicts['id'] = result
    return result


In [238]:
class BaseSheet():

  def __init__(self, sheet):
    self.sheet = sheet

  def clean_cell(self, row, index, field_name):
    try:
      if pd.isnull(row[field_name]):
        return ""
      else:
        return str(row[field_name]).strip()
    except Exception as e:
      print("Clean cell error (%s) for field '%s' in row %s" % (e, field_name, index + 1))
      return ""

  def clean_cell_unnamed(self, rindex, cindex):
    try:
      if pd.isnull(self.sheet.iloc[rindex, cindex]):
        return ""
      else:
        return self.sheet.iloc[rindex, cindex].strip()
    except Exception as e:
      print("Clean cell unnamed error (%s) for cell [%s, %s]" % (e, rindex + 1, cindex + 1))
      return ""

  def cdisc_code_cell(self, row, index, field_name):
    parts = self.clean_cell(row, index, field_name).split("=")
    return self.cdisc_code(code=parts[0], decode=parts[1])

  def cdisc_code(self, code, decode):
    json = DDFJson()
    return json.add_code(code=code, code_system="http://www.cdisc.org", code_system_version="2022-03-25", decode=decode)

  def double_link(self, items, id, prev, next):
    #print("DL1", items, id, prev, next)
    for idx, item in enumerate(items):
      if idx == 0:
        print("DL2", item, prev, item.keys())
        if prev in item:
          print("DL3", item[prev])
        item[prev] = None
      else:
        uuid = items[idx-1][id]
        item[prev] = uuid
      if idx == len(items)-1:  
        item[next] = None
      else:
        uuid = items[idx+1][id]
        item[next] = uuid


In [239]:
class StudyIdentifiersSheet(BaseSheet):

  def __init__(self, file_path):
    try:
      super().__init__(pd.read_excel(open(file_path, 'rb'), sheet_name='studyIdentifiers'))
      self.identifiers = []
      self.process_sheet()
    except Exception as e:
      print("Oops!", e, "occurred.")
      traceback.print_exc()
      
  def process_sheet(self):
    json = DDFJson()
    self.identifiers = []
    for index, row in self.sheet.iterrows():
      organisation_type_key = self.clean_cell(row, index, 'organisationType')
      if organisation_type_key.lower() == "sponsor":
        organisation_type = self.study_sponsor()
      elif organisation_type_key.lower() == "registry":
        organisation_type = self.study_registry()
      elif organisation_type_key.lower() == "regulatory":
        organisation_type = self.regulatory()
      else:
        organisation_type = self.study_sponsor()
      organisation = json.add_organisation(
        scheme=self.clean_cell(row, index, 'organisationIdentifierScheme'), 
        identifier=self.clean_cell(row, index, 'organisationIdentifier'),
        name=self.clean_cell(row, index, 'organisationName'),
        type=organisation_type
      )
      self.identifiers.append(json.add_study_identifier(identifier=self.clean_cell(row, index, 'studyIdentifier'), scope=organisation))
    
  def study_registry(self):
    return self.cdisc_code(code="C93453", decode="Study Registry")

  def study_sponsor(self):
    return self.cdisc_code(code="C70793", decode="Clinical Study Sponsor")

  def regulatory(self):
    return self.cdisc_code(code="C188863", decode="Regulatory Agency")


In [240]:
class StudyDesignSheet(BaseSheet):

  def __init__(self, file_path):
    try:
      super().__init__(pd.read_excel(open(file_path, 'rb'), sheet_name='studyDesign', header=None))
      self.epochs = []
      self.epoch_map = {}
      self.arms = []
      self.cells = []
      self.study_designs = []
      self.process_sheet()
    except Exception as e:
      print("Oops!", e, "occurred.")
      traceback.print_exc()

  def process_sheet(self):
    json = DDFJson()
    #print("COLS", len(self.sheet.columns))
    for rindex, row in self.sheet.iterrows():
      for cindex in range(0, len(self.sheet.columns)):
        cell = self.clean_cell_unnamed(rindex, cindex)
        #print("CELL [%s,%s] %s" % (rindex, cindex, cell))
        if rindex == 0:
          if cindex != 0:
            epoch = json.add_study_epoch(name=cell, description=cell)
            self.epoch_map[cell] = epoch
            self.epochs.append(epoch)
        else:
          if cindex == 0:
            self.arms.append(json.add_study_arm(name=cell, description=cell))
          else:
            self.cells.append(json.add_study_cell(arm=self.arms[-1], epoch=self.epochs[cindex-1]))

    self.double_link(self.epochs, 'studyEpochId', 'previousStudyEpochId', 'nextStudyEpochId')
    trial_type = self.cdisc_code('C12345', 'Observational')
    int_model = self.cdisc_code('C12346', 'None')
    study_design = json.add_study_design(cells=self.cells, intent_types=[], trial_types=[trial_type], intervention_model=int_model)
    #print("STUDY_DESIGN:", study_design)
    self.study_designs.append(study_design)

  def link_encounters(self, encounter_map):
    for epoch_name, encounters in encounter_map.items():
      #print("LINK: %s %s" % (epoch_name, encounter))
      epoch = self.epoch_map[epoch_name]
      for encounter in encounters:
        epoch.encounters.append(encounter)
  
  def link_wfi(self, wfi):
    #print("WFI:", wfi)
    self.study_designs[0].studyWorkflows = [Workflow(workflowDesc="SoA", workflowItems=wfi)]
    #print("SDWF:", self.study_designs[0].studyWorkflows)


In [241]:
class StudySoASheet(BaseSheet):

  EPOCH_ROW = 0
  CYCLE_ROW = 1
  CYCLE_START_ROW = 2
  CYCLE_PERIOD_ROW = 3
  CYCLE_END_RULE_ROW = 4
  TIMING_ROW = 5
  VISIT_LABEL_ROW = 6
  VISIT_WINDOW_ROW = 7

  HEADER_ROW = 8
  FIRST_ACTIVITY_ROW = 9

  ACTIVITY_COL = 0
  CHILD_ACTIVITY_COL = 1
  BC_COL = 2
  FIRST_VISIT_COL = 3

  def __init__(self, file_path):
    try:
      super().__init__(pd.read_excel(open(file_path, 'rb'), sheet_name='soa', header=None))
      self.timeline = {}
      #self.epoch_encounter_map = {}
      #self.activity_map = {}
      self.row_activities_map = []
      self.activity_bc_map = {}
      self.sheet = self.sheet.fillna(method='ffill', axis=1)
      self.cycles = self.extract_cycles()
      self.timepoints = self.extract_timepoints()
      self.encounters = self.extract_encounters()
      self.activity_bc_map, self.row_activities_map, self.activities = self.extract_activities_and_bcs()
      self.tp_activities = self.extract_timepoint_activities_map()

      self.process_sheet()
    except Exception as e:
      print("Oops!", e, "occurred.")
      traceback.print_exc()

  def get_cycle_cell(self, row_index, col_index):
    is_null = pd.isnull(self.sheet.iloc[row_index, col_index])
    if is_null:
      return "", True
    else:
      value = str(self.sheet.iloc[row_index, col_index])
      if value.upper() == "-":
        return "", True
      else:
        return value, False

  def previous_index(index):
    if index == 0:
      return 0
    else:
      return index - 1

  def build_cycle_record(self, index, col_index, cycle):
    cycle_start_index = index
    cycle_start, is_null = self.get_cycle_cell(self.CYCLE_START_ROW, col_index)
    cycle_period, is_null = self.get_cycle_cell(self.CYCLE_PERIOD_ROW, col_index)
    cycle_end_rule, is_null = self.get_cycle_cell(self.CYCLE_END_RULE_ROW, col_index)
    return { 
      'start_index': cycle_start_index, 
      'cycle': cycle, 
      'start': cycle_start, 
      'period': cycle_period, 
      'end_rule': cycle_end_rule 
    }

  def extract_cycles(self):
    cycles = []
    timepoint_index = -1
    cycle_start_index = None
    in_cycle = False
    prev_cycle = None
    for col_index in range(self.sheet.shape[1]):
      if col_index >= self.FIRST_VISIT_COL:
        timepoint_index += 1
        cycle, cycle_is_null = self.get_cycle_cell(self.CYCLE_ROW, col_index)
        if cycle_is_null:
          if in_cycle:
            cycle_record['end_index'] = self.previous_index(timepoint_index)
            cycles.append(cycle_record)
            in_cycle = False
          else:
            pass # Do nothing
        else:
          cycle = str(cycle)
          if not in_cycle:
            in_cycle = True
            cycle_record = self.build_cycle_record(timepoint_index, col_index, cycle)
          elif prev_cycle == cycle:
            pass # Do nothing
          else:
            cycle_record['end_index'] = self.previous_index(timepoint_index)
            cycles.append(cycle_record)
            cycle_record = self.build_cycle_record(timepoint_index, col_index, cycle)
        prev_cycle = cycle
    return cycles

  def get_timing_cell(self, row_index, col_index):
    is_null = pd.isnull(self.sheet.iloc[row_index, col_index])
    if is_null:
      return "", True
    else:
      return self.sheet.iloc[row_index, col_index], False

  def get_activity_cell(self, row_index, col_index):
    is_null = pd.isnull(self.sheet.iloc[row_index, col_index])
    if is_null:
      return "", True
    else:
      value = self.sheet.iloc[row_index, col_index]
      if value == '-':
        return "", True
      else:
        return self.sheet.iloc[row_index, col_index], False
  
  def get_observation_cell(self, row_index, col_index):
    is_null = pd.isnull(self.sheet.iloc[row_index, col_index])
    if is_null:
      return "", "", True
    else:
      value = self.sheet.iloc[row_index, col_index]
      if value == '-':
        return "", "", True
      else:
        parts = value.split(':')
        if parts[0].lower() == "bc":
          return "bc", parts[1], False
        else:
          return "", "", True

  def get_relative_ref(self, part):
    if len(part) > 1:
      return int(part[1:])
    else:
      return 1

  def get_timing_type(self, col_index):
    timing_type = ""
    rel_ref = 0
    timing_value = ""
    timing_info, timing_info_is_null = self.get_timing_cell(self.TIMING_ROW, col_index)
    if not timing_info_is_null:
      timing_parts = timing_info.split(":")
      if timing_parts[0].upper()[0] == "A":
        timing_type = "anchor"
        rel_ref = 0
      if timing_parts[0].upper()[0] == "P":
        timing_type = "previous"
        rel_ref = self.get_relative_ref(timing_parts[0]) * -1
      elif timing_parts[0].upper()[0] == "N":
        timing_type = "next"
        rel_ref = self.get_relative_ref(timing_parts[0])
      elif timing_parts[0].upper()[0] == "C":
        timing_type = "cycle start"
        rel_ref = self.get_relative_ref(timing_parts[0])
      if len(timing_parts) == 2:
        timing_value = timing_parts[1].strip()
    #print("TIMING: col_index (%s) - FIRST_VISIT_COL (%s) + rel_ref (%s)" % (col_index, FIRST_VISIT_COL, rel_ref))
    return { 'type': timing_type, 'ref': col_index - self.FIRST_VISIT_COL + rel_ref, 'value': timing_value, 'cycle': None }

  def extract_timepoints(self):
    timepoints = []
    for col_index in range(self.sheet.shape[1]):
      if col_index >= self.FIRST_VISIT_COL:
        record = self.get_timing_type(col_index)
        timepoints.append(record)
    return timepoints

  def get_encounter_cell(self, row_index, col_index):
    is_null = pd.isnull(self.sheet.iloc[row_index, col_index])
    if is_null:
      return "", True
    else:
      return self.sheet.iloc[row_index, col_index], False

  def get_encounter_details(self, col_index):
    label = ""
    window = ""
    label, label_is_null = self.get_encounter_cell(self.VISIT_LABEL_ROW, col_index)
    window, window_is_null = self.get_encounter_cell(self.VISIT_WINDOW_ROW, col_index)
    return { 'label': label, 'window': window }

  def extract_encounters(self):
    encounters = []
    for col_index in range(self.sheet.shape[1]):
      if col_index >= self.FIRST_VISIT_COL:
        record = self.get_encounter_details(col_index)
        encounters.append(record)
    return encounters

  def extract_activities_and_bcs(self):
    activity_bc_map = {}
    row_activities_map = []
    activities = []
    prev_activity = None
    for row_index, col_def in self.sheet.iterrows():
      if row_index >= self.FIRST_ACTIVITY_ROW:
        activity, activity_is_null = self.get_activity_cell(row_index, self.CHILD_ACTIVITY_COL)
        if activity_is_null:
          if not prev_activity == None:
            row_activities_map.append(prev_activity)
            activity = prev_activity
        else:
          activities.append(activity)
          row_activities_map.append(activity)
        prev_activity = activity
        obs_type, obs_name, obs_is_null = self.get_observation_cell(row_index, self.BC_COL)
        if not obs_is_null:
          if obs_type == "bc":
            if not activity in activity_bc_map:
              activity_bc_map[activity] = { 'bc': [] }  
            activity_bc_map[activity]['bc'].append(obs_name)
    return activity_bc_map, row_activities_map, activities
  
  def extract_timepoint_activities_map(self):
    timepoint_activity_map = []
    activity_dict = {}
    for activity in self.activities:
      activity_dict[activity] = False
    for tp in self.timepoints:
      timepoint_activity_map.append(dict(activity_dict))
    for index in range(self.sheet.shape[1]):
      if index >= self.FIRST_VISIT_COL:
        column = self.sheet.iloc[:, index]
        row = 0
        for col in column:
          if row >= self.FIRST_ACTIVITY_ROW:
            if not pd.isnull(col):
              if col.upper() == "X":
                print("RA", self.row_activities_map, row)
                activity = self.row_activities_map[row - self.FIRST_ACTIVITY_ROW]
                tp_index = index - self.FIRST_VISIT_COL
                timepoint_activity_map[tp_index][activity] = True
          row += 1
    return timepoint_activity_map

  def process_sheet(self):
  #def process_timepoints(self, timepoints, cycles, activities, tp_activities, encounters):
    json = DDFJson()
    tps = []
    acts = []
    encs = []
    bcs = []
    acts_map = {}
    timing = []
    cycle_offset = 0
    for index, timepoint in enumerate(self.timepoints):
      timepoint['activity_index'] = index
      timepoint['encounter_index'] = index
    for cycle in self.cycles:
      start_index = cycle['start_index'] + cycle_offset
      self.timepoints.insert(start_index, { 'type': 'anchor', 'ref': 0, 'value': cycle['start'], 'activity_index': None, 'encounter_index': None, 'cycle': cycle['cycle'] })
      cycle_offset += 1
      end_index = cycle['end_index'] + cycle_offset + 1
      self.timepoints.insert(end_index, { 'type': 'previous', 'ref': end_index - 1, 'value': cycle['period'], 'activity_index': None, 'encounter_index': None, 'cycle': None })
      cycle_offset += 1
      end_index = cycle['end_index'] + cycle_offset + 1
      self.timepoints.insert(end_index, { 'type': 'condition', 'ref': start_index , 'value': cycle['end_rule'], 'activity_index': None, 'encounter_index': None, 'cycle': None })
      cycle_offset += 1
    previous_tp_id = None
    for activity in self.activities:
      a_bcs = []
      if activity in self.activity_bc_map:
        for a in self.activity_bc_map[activity]['bc']:
          bc = json.add_biomedical_concept_surrogate(a, a, "")
          a_bcs.append(bc['bcSurrogateId'])
          bcs.append(bc)
      acts.append(json.add_activity(activity, activity, False, "", a_bcs))
      acts_map[activity] = acts[-1]['activityId']
    for encounter in self.encounters:
      encs.append(json.add_encounter(encounter['label'], encounter['label'], None, None, []))
    for timepoint in self.timepoints:
      activity_ids = []
      encounter_id = None
      if not timepoint['activity_index'] == None:
        source = self.tp_activities[timepoint['activity_index']]
        for k, v in source.items():
          if v:
            activity_ids.append(acts_map[k])
      if not timepoint['encounter_index'] == None:
        encounter_id = encs[timepoint['encounter_index']]['encounterId']
      tps.append(json.add_timepoint(previous_tp_id, None, activity_ids, encounter_id))
      previous_tp_id = tps[-1]['timepointId']
    for index, timepoint in enumerate(self.timepoints):
      if timepoint['type'] == 'condition':
        tps[index]['cycleId'] = tps[timepoint['ref']]['timepointId']
        tps[index]['_type'] = 'Condition'
    for timepoint in self.timepoints:
      if timepoint['type'] == 'next':
        timing.append(json.add_next_timing(timepoint['value'], 'StartToStart', None, tps[timepoint['ref']]['timepointId']))
      elif timepoint['type'] == 'previous':
        timing.append(json.add_previous_timing(timepoint['value'], 'StartToStart', None, tps[timepoint['ref']]['timepointId']))
      elif timepoint['type'] == 'anchor':
        timing.append(json.add_anchor_timing(timepoint['value'], timepoint['cycle']))
      elif timepoint['type'] == 'condition':
        #timing.append(json.add_condition_timing(timepoint['value']))
        timing.append({})
      elif timepoint['type'] == 'cycle start':
        timing.append(json.add_cycle_start_timing(timepoint['value']))
      elif timepoint['type'] == '':
        timing.append({})
    for index, tp in enumerate(tps):
      tp['scheduledAt'] = timing[index]
    entry = json.add_entry('Main timeline', tps[0]['timepointId'])
    exit = json.add_exit()
    tps[-1]['exit'] = exit
    self.timeline = json.add_timeline(entry, tps, exit)
    
    # def export(self, node):
    #   return self.export_node(node)

    # def export_node(self, node):
    #   if type(node) == list:
    #     result = []
    #     for item in node:
    #       result.append(self.export_node(item))
    #     return result
    #   elif type(node) == dict:
    #     result = {}
    #     for key, value in node.items():
    #       if key.startswith('_'):
    #         continue
    #       result[key] = self.export_node(value)
    #     return result
    #   else:
    #     return node

  # def process_sheet(self):
  #   #print("SIZE %s x %s" % (self.sheet.shape[0], len(self.sheet.columns)))
  #   wfi_index = 1
  #   for rindex, row in self.sheet.iterrows():
  #     for cindex in range(0, len(self.sheet.columns)):
  #       #print("A %s %s" % (rindex, cindex))  
  #       cell = self.clean_cell_unnamed(rindex, cindex)
  #       #print("CELL [%s,%s] %s" % (rindex, cindex, cell))
  #       if rindex == 0:
  #         pass
  #       elif rindex == 1:
  #         if cindex != 0:
  #           epoch = self.clean_cell_unnamed(rindex - 1, cindex)
  #           description = self.clean_cell_unnamed(rindex + 1, cindex)
  #           encounter = Encounter(uuid=str(uuid4()), encounterName=cell, encounterDesc=description)
  #           if not epoch in self.epoch_encounter_map:
  #             self.epoch_encounter_map[epoch] = []
  #           self.epoch_encounter_map[epoch].append(encounter)
  #           self.encounters.append(encounter)
  #       elif rindex == 2:
  #         pass
  #       else:
  #         if cindex == 0:
  #           activity = Activity(uuid=str(uuid4()), activityName=cell, activityDesc=cell)
  #           self.activity_map[cell] = activity
  #           self.activities.append(activity)
  #         else:
  #           if cell.lower() == "x":
  #             self.workflow_items.append(WorkflowItem(uuid=str(uuid4()), workflowItemDesc="WFI%s" % (wfi_index), workflowItemActivity=self.activities[-1], workflowItemEncounter=self.encounters[cindex-1]))
  #             wfi_index += 1
  #   self.double_link(self.activities, 'uuid', 'previousActivityId', 'nextActivityId')
  #   self.double_link(self.encounters, 'uuid', 'previousEncounterId', 'nextEncounterId')
  #   self.double_link(self.workflow_items, 'uuid', 'previousWorkflowItemId', 'nextWorkflowItemId')

  # def link_study_data(self, study_data_map):
  #   for activity_name, study_data in study_data_map.items():
  #     activity = self.activity_map[activity_name]
  #     activity.studyDataCollection = study_data


In [242]:
class StudySheet(BaseSheet):

  def __init__(self, file_path):
    try:
      super().__init__(pd.read_excel(open(file_path, 'rb'), sheet_name='study'))
      self.study = None
      self.study_identifiers = StudyIdentifiersSheet(file_path)
      self.study_design = StudyDesignSheet(file_path)
      self.soa = StudySoASheet(file_path)
      #self.activities = StudyActivitiesSheet(file_path)
      #self.soa.link_study_data(self.activities.activity_map)
      #self.study_designs.link_encounters(self.soa.epoch_encounter_map)
      #self.study_designs.link_wfi(self.soa.workflow_items)
      print("Study 1", self.soa.timeline)
      self.process_sheet()
      print("Study 2")
    except Exception as e:
      print("Oops!", e, "occurred.")
      traceback.print_exc()

  def process_sheet(self):
    json = DDFJson()
    for index, row in self.sheet.iterrows():
      study_phase = self.cdisc_code_cell(row, index, "studyPhase")
      study_version = self.clean_cell(row, index, "studyVersion")
      study_type = self.cdisc_code_cell(row, index, "studyType")
      study_title = self.clean_cell(row, index, "studyTitle")
      self.study = json.add_study(
        title=study_title,
        version=study_version,
        type=study_type,
        phase=study_phase,
        ta=None,
        rationale="",
        acronym="",
        identifiers=self.study_identifiers.identifiers,
        protocols=[],
        designs=[]
      )

  def study_sponsor(self):
    return self.cdisc_code(code="C93453", decode="Study Registry")

  def study_regulatory(self):
    return self.cdisc_code(code="C93453", decode="Study Registry")

  def the_study(self):
    return self.study

In [243]:
class ImportExcel():

  def __init__(self, file_path):
    self.study = StudySheet(file_path)

  def identifier(self):
    study = self.study.the_study()
    if study == None:
      return None
    else:
      return study.study_identifier()

  def save(self):
    return self.study.study.save()

  def the_study(self):
    return self.study.the_study()

In [244]:
class DDFVisual():

  def __init__(self):
    self.nodes = []
    self.edges = []
    self.add_edges = []
    self.node_index = 1
    self.edge_index = 1
    self.id_node_index_map = {}
    self.type_id_field_map = { 
      'Entry': 'entryId',
      'Exit': 'exitId',
      'Timeline': 'timelineId',
      'Timepoint': 'timepointId',
      'Timing': 'timingId',
      'Condition': 'timepointId',
      'CycleStart': 'cycleStartId',
      'StudyDesign': 'studyDesignId',
      'Activity': 'activityId',
      'Encounter': 'encounterId',
      'BCSurrogate': 'bcSurrogateId',
      'Study': 'studyId'
    }
    self.edge_attributes = [
      'relativeTo',
      'nextTimepointId',
      'cycleId',
      'timepointActivityIds',
      'timepointEncounterId',
      'bcSurrogateIds',
      'bcCategoryIds',
      'biomedicalConceptIds',
    ]
    
  def get_id_field_and_klass(self, node):
    klass = node['_type']
    return self.type_id_field_map[klass], klass

  def draw(self, json):
    self.process_node(json)
    for edge in self.add_edges:
      if edge['end'] in self.id_node_index_map:
        edge['id'] = self.edge_index
        edge['end'] = self.id_node_index_map[edge['end']]
        self.edges.append(edge)
        self.edge_index += 1
      else:
        print("***** %s -edge-> %s *****" % (edge['start'], edge['end']))
    return self.nodes, self.edges
  
  def process_node(self, node):
    if type(node) == list:
      result = []
      for item in node:
        indexes = self.process_node(item)
        result = result + indexes
      return result
    elif type(node) == dict:
      if node == {}:
        return []
      properties = {}
      id_field, klass = self.get_id_field_and_klass(node)
      if node[id_field] in self.id_node_index_map:
        return [self.id_node_index_map[node[id_field]]]
      this_node_index = self.node_index
      self.node_index += 1
      for key, value in node.items():
        if key in self.edge_attributes:
          if type(value) == list:
            for item in value:
              self.add_edges.append( { 'start': this_node_index, 'end': item, 'properties': {'label': key}})
          else:
            self.add_edges.append( { 'start': this_node_index, 'end': value, 'properties': {'label': key}})
        else:
          indexes = self.process_node(value)
          if indexes == []:
            properties[key] = value
          else:
            for index in indexes:
              self.edges.append( {'id': self.edge_index, 'start': this_node_index, 'end': index, 'properties': {'label': key}})
              self.edge_index += 1
      properties['node_type'] = klass
      properties['label'] = node[id_field]
      self.nodes.append({ 'id': this_node_index, 'properties': properties })
      self.id_node_index_map[properties[id_field]] = this_node_index
      return [this_node_index]
    else:
      return []

In [245]:
def save_as_file(data, filename):
  with open('source_data/%s.json' % (filename), 'w', encoding='utf-8') as outfile:
    json.dump(data, outfile, indent=2)

In [246]:
#study = "Roche Phase 3 NCT04320615"
#study = "cycles_1_v2"
study = "simple_1"
#study = "simple_2"

notebook_path = os.path.abspath("notebook.ipynb")
file_path = os.path.join(os.path.dirname(notebook_path), "source_data/%s.xlsx" % (study))
x = ImportExcel(file_path)
print(x.the_study())


#print("CYCLES", cycles)
#print("TIMEPOINTS", timepoints)
#print("ENCOUNTERS", encounters)
#print("ACTIVITIES", activities)
#print("TP ACTIVITIES", tp_activities)

#x = DDFJson()
#node = x.process_timepoints(timepoints, cycles, activities, tp_activities, encounters)
#save_as_file(x.export(node), study)

#y = DDFVisual()
#nodes, edges = y.draw(node)


DL2 {'_type': 'StudyEpoch', 'studyEpochId': 'study_epoch_1', 'studyEpochName': 'Screening', 'studyEpochDescription': 'Screening', 'studyEpochType': '', 'previousStudyEpochId': '', 'nextStudyEpochId': '', 'encounters': []} previousStudyEpochId dict_keys(['_type', 'studyEpochId', 'studyEpochName', 'studyEpochDescription', 'studyEpochType', 'previousStudyEpochId', 'nextStudyEpochId', 'encounters'])
DL3 
RA ['Demographics', 'Demographics', 'Demographics', 'Something Else'] 9
RA ['Demographics', 'Demographics', 'Demographics', 'Something Else'] 12
RA ['Demographics', 'Demographics', 'Demographics', 'Something Else'] 12
RA ['Demographics', 'Demographics', 'Demographics', 'Something Else'] 12
RA ['Demographics', 'Demographics', 'Demographics', 'Something Else'] 12
RA ['Demographics', 'Demographics', 'Demographics', 'Something Else'] 12
Study 1 {'_type': 'Timeline', 'timelineId': 'timeline_1', 'timelineEntry': {'_type': 'Entry', 'entryId': 'entry_1', 'entryDescription': 'Main timeline', 'nextT

In [247]:

def custom_node_color(index: int, node: dict):
  if 'node_type' in node['properties']:
    if node['properties']['node_type'] == 'Entry':
      return 'black'
    elif node['properties']['node_type'] == 'Exit':
      return 'black'
    elif node['properties']['node_type'] == 'Timeline':
      return '#3F6AFC'
    elif node['properties']['node_type'] == 'Condition':
      return '#ABB2B9'
    elif node['properties']['node_type'] == 'CycleStart':
      return '#3F6AFC'
    elif node['properties']['node_type'] == 'Timing':
      return '#6495ED'
    elif node['properties']['node_type'] == 'Timepoint':
      return '#ABB2B9'
    elif node['properties']['node_type'] == 'Activity':
      return '#1BA62F'
    elif node['properties']['node_type'] == 'BCSurrogate':
      return '#1BA62F'
    elif node['properties']['node_type'] == 'Encounter':
      return '#E53F2F'
    elif node['properties']['node_type'] == 'StudyDesign':
      return '#E52FDA'
    elif node['properties']['node_type'] == 'Study':
      return '#E52FDA'
    else:
      return 'white'
  else: 
    return 'white'

def custom_node_style(index: int, node: dict):
  if 'node_type' in node['properties']:
    if node['properties']['node_type'] == 'Entry' or node['properties']['node_type'] == 'Exit':
      return {'image': 'https://raw.githubusercontent.com/data4knowledge/timepoints/main/images/pill_black.svg'}
    elif node['properties']['node_type'] == 'Timeline':
      return { 'shape': 'ellipse' }
    elif node['properties']['node_type'] == 'Timing':
      return { 'shape': 'ellipse' }
    elif node['properties']['node_type'] == 'Condition':
      return { 'shape': 'diamond' }
    elif node['properties']['node_type'] == 'CycleStart':
      return { 'shape': 'hexagon2' }
    elif node['properties']['node_type'] == 'Timepoint':
      return { 'shape': 'ellipse' }
    elif node['properties']['node_type'] == 'Activity':
      return { 'shape': 'ellipse' }
    else:
      return { 'shape': 'ellipse' }
  else: 
    return { 'shape': 'ellipse' }

graph_widget = GraphWidget()
graph_widget.orthogonal_layout()
graph_widget.set_directed(True)

graph_widget.set_nodes(nodes)
graph_widget.set_edges(edges)
graph_widget.set_node_color_mapping(custom_node_color)
graph_widget.set_node_styles_mapping(custom_node_style)
graph_widget

NameError: name 'nodes' is not defined